<a href="https://colab.research.google.com/github/giselesousar/msr_tests/blob/main/tests_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [233]:
!pip install pydriller

     |████████████████████████████████| 65 kB 2.1 MB/s 
     |████████████████████████████████| 180 kB 8.7 MB/s 
     |████████████████████████████████| 64 kB 3.1 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 


In [237]:
import os
import json
from pathlib import Path
from enum import Enum
from pydriller import Repository

# Global variables

In [238]:
global repository_name
global repository_url
repository_name = 'promocity'
repository_url = 'https://github.com/armandossrecife/promocity.git'

class Type(Enum):
  DIR = 0
  FILE = 1

# Clone repository

In [239]:
![ -d $repository_name ] && echo "Remove diretório antigo" && rm -R $repository_name
!git clone $repository_url

!sudo apt install tree

Remove diretório antigo
Cloning into 'promocity'...
remote: Enumerating objects: 841, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 841 (delta 10), reused 21 (delta 2), pack-reused 798
Receiving objects: 100% (841/841), 4.00 MiB | 13.65 MiB/s, done.
Resolving deltas: 100% (324/324), done.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tree is already the newest version (1.7.0-5).
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-

# Analizer

In [240]:
# Lista de nomes de arquivos ou diretórios para serem ignorados
list_of_files_and_directories_to_ignore = ['.git']

# dada uma string, retorna true se pelo menos um dos arquivos ou diretórios
# que devem ser ignorados aparecem nela
def should_ignore(name):
  return True in [i in each for i in list_of_files_and_directories_to_ignore]

In [241]:
# Cria um arquivo contendo a quantidade de LOC por arquivo
!find $repository_name | xargs wc -l > locarquivos.txt
list_locs_of_files = !cat locarquivos.txt

wc: promocity: Is a directory
wc: promocity/docs: Is a directory
wc: promocity/docs/Promocity: No such file or directory
wc: Um: No such file or directory
wc: serviço: No such file or directory
wc: ubíquo: No such file or directory
wc: geolocalizado: No such file or directory
wc: de: No such file or directory
wc: promoções: No such file or directory
wc: contextuais.pdf: No such file or directory
wc: promocity/docs/Statemachine: No such file or directory
wc: Diagram: No such file or directory
wc: Checa: No such file or directory
wc: Proximidade: No such file or directory
wc: Usuario.asta: No such file or directory
wc: promocity/docs/Statemachine: No such file or directory
wc: Contexto: No such file or directory
wc: Proximidade.png: No such file or directory
wc: promocity/docs/Diagrama: No such file or directory
wc: de: No such file or directory
wc: Classes: No such file or directory
wc: Servicos: No such file or directory
wc: Promocity.asta: No such file or directory
wc: promocity/d

In [242]:
# calcula frequência dos arquivos em commits
# TODO: otimizar forma de iterar dos commits

list_of_files_frequency_in_commits = {}

commits = Repository(path_to_repo=repository_name).traverse_commits()

for commit in commits:
    for modified_file in commit.modified_files:
        if modified_file.filename in list_of_files_frequency_in_commits.keys():
            list_of_files_frequency_in_commits[modified_file.filename] += 1
        else:
            list_of_files_frequency_in_commits[modified_file.filename] = 1

In [245]:
# cria dicionário com o path de cada arquivo e LOC, ignorando aqueles que 
# não devem aparecer no json
list_locs_of_files_updated = {}
for each in list_locs_of_files[:-1]:
  if not should_ignore(each):
    elementos = each.split(' ') 
    list_locs_of_files_updated[elementos[-1]] = int(elementos[-2])

In [246]:
# Lista todos os arquivos e diretorios
list_of_files_and_directories = !cd $repository_name && tree -i -f

In [247]:
# Substitui o . pelo nome do repositório
list_of_files_and_directories = [each.replace('./', repository_name + '/') for each in list_of_files_and_directories][1:-2]

In [248]:
# remove os arquivos que devem ser ignorados
list_of_files_and_directories_updated = []

for each in list_of_files_and_directories:
  if not should_ignore(each):
    list_of_files_and_directories_updated.append(each)

In [249]:
# classe que representa cada nó da árvore
class Node:
  def __init__(self, name = None, loc = None, heatmap = None, node_type = None, depth = None, children = []):
    self.name = name
    self.loc = loc
    self.node_type = node_type
    self.depth = depth
    self.children = children
    self.parent = None
    self.heatmap = heatmap
  
  def append_node(self, node):
    node.parent = self
    self.children.append(node)

In [256]:
# preenchendo árvore com os arquivos e diretórios

root = Node(name=repository_name, loc=0, heatmap=0, node_type=Type.DIR, depth=0, children=[])

# lista auxiliar para controlar a profundidade da árvores
# os nós são criados e inseridos como filhos do nó que está na última posição da lista
nodes = [root]

for key in list_of_files_and_directories_updated:
  name = key.split('/')[-1] 
  depth = key.count('/')
  loc = 0
  heatmap = 0

  if os.path.isdir(key):
    key_type = Type.DIR
    parent = os.path.dirname(key).split('/')[-1]
  else:
    key_type = Type.FILE
    parent = str(Path(key).parent).split('/')[-1]
    if key in list_locs_of_files_updated:
      loc = list_locs_of_files_updated[key]
    if name in list_of_files_frequency_in_commits:
      heatmap = list_of_files_frequency_in_commits[name]
  
  # controle do último nó da lista auxiliar (onde deve ser inserido o nó atual)
  if depth != len(nodes):
    diff = abs(depth - len(nodes))
    if depth < len(nodes):
      for i in range(0, diff):
        nodes.pop()
    else:
      nodes.append(node)

  node = Node(name=name, loc=loc, heatmap=heatmap, node_type=key_type, depth=depth, children=[])

  if len(nodes) > 0:
    nodes[len(nodes) - 1].append_node(node)

In [257]:
# atualizar LOC de toda a árvore com base nas folhas
def calculate_loc(node):
  loc = 0
  for each in node.children:
    loc += each.loc + calculate_loc(each)
  if len(node.children) > 0:
    node.loc = loc
  if node.parent is not None:
    return loc

# atualizar métrica do heatmap de toda a árvore com base nas folhas
def calculate_heatmap(node):
  heatmap = 0
  for each in node.children:
    heatmap += each.heatmap + calculate_heatmap(each)
  if len(node.children) > 0:
    node.heatmap = heatmap
  if node.parent is not None:
    return heatmap

calculate_loc(root)
calculate_heatmap(root)

In [258]:
# mapeia árvore criada para o formato JSON

def create_json_node(node):
  return {
    "name": node.name,
    "type": node.node_type.name,
    "loc": node.loc,
    "depth": node.depth,
    "heatmap": node.heatmap,
    "children": []
  }

def traverse(node):
  children = []
  for each in node.children:
    node = create_json_node(each)
    children.append(node)
    node["children"] = traverse(each)
  return children

json_output = create_json_node(root)
json_output["children"] = traverse(root)

print(json.dumps(json_output))

{"name": "promocity", "type": "DIR", "loc": 12392, "depth": 0, "heatmap": 216, "children": [{"name": "docs", "type": "DIR", "loc": 4270, "depth": 1, "heatmap": 12, "children": [{"name": "Diagrama de Classes Servicos Promocity.asta", "type": "FILE", "loc": 0, "depth": 2, "heatmap": 1, "children": []}, {"name": "Diagrama de Classes Servicos Promocity.png", "type": "FILE", "loc": 0, "depth": 2, "heatmap": 1, "children": []}, {"name": "index.html", "type": "FILE", "loc": 3191, "depth": 2, "heatmap": 5, "children": []}, {"name": "Promocity - Um servic\u0327o ubi\u0301quo geolocalizado de promoc\u0327o\u0303es contextuais.pdf", "type": "FILE", "loc": 0, "depth": 2, "heatmap": 1, "children": []}, {"name": "Statemachine Contexto Proximidade.png", "type": "FILE", "loc": 0, "depth": 2, "heatmap": 1, "children": []}, {"name": "Statemachine Diagram Checa Proximidade Usuario.asta", "type": "FILE", "loc": 0, "depth": 2, "heatmap": 1, "children": []}, {"name": "swagger-5.yaml", "type": "FILE", "loc":